# Prime numbers

Prime numbers are of crucial importance in cryptography. This is why it is important to learn their properties and how to generate them in a fast and reliable way.


# Table of contents:

* [Prime numbers definition](#vigenere-revisited)
* [The fundamental theorem of arithmetics](#fta)
* [Prime numbers density](#primes-density)
* [How to find prime numbers](#primes-find)
    * [Sieve of Erathosthenes](#eratos)
    * [Fermat's little theorem and inverse modulo $p$](#fermat)
    * [Miller-Rabin primality testing](#miller)
    
Author: [Sebastià Agramunt Puig](https://github.com/sebastiaagramunt) for [OpenMined](https://www.openmined.org/) Privacy ML Series course.


## Prime numbers definition <a class="anchor" id="primes"></a>

**Definition**: A prime number is a number that is only divisible by himself and 1. Equivalently $p$ is a prime number if gcd($a$, $p$)=1 for all $a$ smaller than $p$. The first prime numbers are

2, 3, 5, 7, 11, 13, 17...

Some facts about prime numbers:

* There are infinite number of prime numbers
* There is no formula to calculate the nth prime number

## Fundamental theorem of arithmetics <a class="anchor" id="fta"></a>

The fundamental theorem of arithmetics states that: **every integer greater than 1 either is a prime number itself or can be represented as the product of prime numbers**. Furthermore this representation is unique

$$a=p_1^{e_1}p_2^{e_2}\cdots p_r^{e_r}$$

where $p$s are prime numbers and $e$s the exponentiation of them. E.g. $5568=2^6*3*29$, where 2, 3 and 29 are prime numbers. 


## Prime number density <a class="anchor" id="primes-density"></a>

Even though we can’t predict primes with a formula we can calculate the probability of finding a prime number in between a range of numbers. We define $\pi$(x) as the number of prime numbers smaller than x. For instance, $\pi$(20)=8, because prime numbers smaller than 20 are {2, 3, 5, 7, 11, 13, 17, 19}. Therefore, to calculate the number of primes between $x_2$ and $x_1$ ($x_2$>$x_1$) we just need to subtract both $\pi$($x_2$)-$\pi$($x_1$). Here $\pi$(x) is exact calculation and we have to do it numerically. The prime number theorem establishes the asymptotic distribution of prime numbers by approximating the count to x/ln(x)

<img src="img/prime_number_density.png" style="width:500px;height500px"/>

## Finding prime numbers <a class="anchor" id="primes-find"></a>

In this section we aim to generate prime numbers. First a list of the first $N$ prime numbers and then arbitrarily large prime numbers (in theory!).


### The sieve of Eratosthenes <a class="anchor" id="eratos"></a>

How can we calculate a list of primes that are smaller than $n$?. We will use the Sieve of Erathostenes: build a list of all the natural numbers from 1 to n (n is the natural number below which you want all the primes) and remove all the multiples of the newly find prime.

Say we want the prime numbers smaller than n=120. The algorithm starts with 2, then you discard its multiples (2, 4, 8, …, 120), you go for 3 and you know it is prime because it hasn’t been discarded, so you eliminate multiples of 3 that haven’t been discarded (3, 9, 12, 15…). The next number is 4 and has been discarded so you go to 5 and add it as prime, then eliminate its multiples…

In [1]:
def PrimesSieveEratosthenes(limit: int):
    a = [True]*limit
    a[0] = a[1] = False
    for i, isprime in enumerate(a):
        if isprime:
            yield i
            for n in range(i*i, limit, i):
                a[n] = False
        
list(PrimesSieveEratosthenes(100))

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97]

In [2]:
from numpy import log

n2 = 1000000
n1 = 100000

def expectedPrimes(x):
    return x/log(x)

print(f"Expected number of primes between {n2} and {n1} is {expectedPrimes(n2)-expectedPrimes(n1)}")

Expected number of primes between 1000000 and 100000 is 63696.52401247693


In [3]:
all_primes = list(PrimesSieveEratosthenes(n2))

primes_between = []
for prime in all_primes:
    if prime < n2 and prime > n1:
        primes_between.append(prime)
        
print(f"Number of primes between {n2} and {n1} is {len(primes_between)}")

Number of primes between 1000000 and 100000 is 68906


A randomly chosen number $n$ has a probability $1/\log n$ of being prime

### Fermat's little theorem <a class="anchor" id="fermat"></a>

Let $p$ be a prime number and let $a$ be any integer, then

$$a^{p}=a\textrm{ (mod p)}$$

where $a^{p}$ means multiplying $a$, $p$ times and apply modulo $p$.

#### Using Fermat's little theorem to find modulo p inverse

In the group of integers 0, 1, ... $p$-1 with multiplication modulo $p$ we can easily find the inverse with Fermat's little theorem:

$$a^{p}=a\textrm{ (mod p)}$$

We just need to multiply by the inverse to the left and to the right of the equation two times

$$a^{p-2}=a^{-1}\textrm{ (mod p)}$$

so the inverse is:

$$a^{-1}=a^{p-2}\textrm{ (mod p)}$$

In [4]:
def InverseFermat(a: int, p: int) -> int:
    # look out, p has to be prime and we are not checking it here!
    return pow(a, p-2, p)

InverseFermat(4, 7)

2

In [5]:
print(f"{4*2%7}")

1


### Primality testing: Miller Rabin algorithm <a class="anchor" id="miller"></a>

The Fermat's theorem to find the inverse modulo $p$ only works when $p$ is prime, we need a way to predict if a number is prime or not in a fast way. Also, there are many cases in which we will be given very large numbers so calculate all of them with the Sieve of Erathostenes is not practical. Here we need a set of algorithms that "test" if a number is prime or not.


### First attempt

We know that $a^{p}=a\textrm{ (mod p)}$ if $p$ is prime. Can we say that $p$ is prime if the previous equation holds for all the possible values of $a<p$?. There are numbers known as **Carmichael numbers** that are composite numbers for which the equation always holds. For instance, 561,41041 and 825265 are the first three Carmichael numbers. Lets see if the condition $a^{p}=a\textrm{ (mod p)}$ is met:


In [6]:
#n = 561
n = 41041
for a in range(1, n):
    if pow(a, n, n)!=a:
        print(a, pow(a, n, n))
        break

In [7]:
# these are composite numbers (not prime):
print(f"Remainder of {561} divided by {3} is {561%3}")
print(f"Remainder of {41041} divided by {11} is {41041%11}")
print(f"Remainder of {825265} divided by {5} is {825265%5}")

Remainder of 561 divided by 3 is 0
Remainder of 41041 divided by 11 is 0
Remainder of 825265 divided by 5 is 0


**If $a^{p}=a\textrm{ (mod p)}$ is met**, for a particular $p$ that **does not mean that $p$ is a prime number**

### Second attempt

Even though $p$ accomplishing the equation of Fermat's little theorem does not mean $p$ is prime, is a good indicative that $p$ may be prime. After all if the Fermat's little equation is not fullfilled we know for sure that the number we are dealing with is not prime. That's what we are going to exploit in our seccond attempt

Let $p$ be a prime number different from 2, then we can write

$$p-1=2^kq$$

where $q$ is an odd number and $k$ an integer. We can do this because we know that 2 is the only even prime number and therefore $p$-1 must be even if the prime number is larger than 2. For $p-1$ being even we can factor it as a product of $k$ times 2, times an even number.

Let $a$ be a number not divisible by $p$, then **one** of the following two conditions is true:

$$a^q-1=0\textrm{ (mod p)}$$
$$a^{cq}+1=0\textrm{ (mod p)}$$

for $c$ in $0, 1, \cdots,2^k$. You can find a proof of this proposition in the [book](https://www.springer.com/gp/book/9781441926746) of Hoffstein, Pipher and Silverman. This happens strictly when $p$ is prime, let's try to see if these equations are true for arbitrary integer $n$.

We take

$$n-1=2^kq$$

If we find an $a$ for which **the two conditions** above are not true, then for sure $n$ is a composite number. $a$ is called a witness for the compositneess of $n$. This is what we do in the Miller-Rabin primality testing:

The Miller Rabin primality testing inputs an integer $n$ for which we want to test if it's prime or not and a potential composite witness $a$. If the two equations above are not true, then we know for sure that $n$ is composite.

1. If $n$ is even or gcd(a, n)!=1, $n$ is composite
2. Find $k$ and $q$ such that $n-1=2^kq$
3. Set $a=a^q$(mod $n$)
4. If $a=1$(mod $n$) Test fails
5. Loop for i=0,1,2,..., $k$-1:
   - If a==-1 (mod $n$) test Fails
   - Set $a=a^2$(mod $n$)
6. End i loop
7. Return Prime

In [8]:
def xgcd(a, b):
    # Solving equation au+bv=gcd(a,b)
    # result is: (g,u,v) where g is greatest common divisor and u,v the solution of the eq above
    u0, u1, v0, v1 = 0, 1, 1, 0
    while a != 0:
        q, b, a = b // a, a, b % a
        v0, v1 = v1, v0 - q * v1
        u0, u1 = u1, u0 - q * u1
    return b, u0, v0

In [9]:
def isWitness(a: int, n: int, q: int, k: int) -> bool:
    x = pow(a, q, n)
    if x==1:
        return False
    for _ in range(k):
        if (x+1)%n == 0:
            return False
        x = pow(x, 2, n)
    return True

In [10]:
from random import randrange

def isPrime(n: int, r: int) -> bool:
    # Miller-Rabin primality testing.
    # n: number to test primality
    # r: times to run the test
    if n<2:
        return False
    if n==2:
        return True
    if n%2==0:
        return False
    
    
    q = n-1
    k = 0
    while q%2 == 0:
        q = q//2
        k += 1
        
    assert n-1==pow(2, k)*q
    
    for _ in range(r):
        a = randrange(2, n)
        if isWitness(a, n, q, k):
            return False
    return True
    
isPrime(150199053749, 10)

True

How sure we are that the number tested is prime?

Proposition: Let $n$ be an odd composite number, then at least 75% of the numbers between 1 and $n$-1 are Miller-Rabin **witnesses** for $n$. Proof in the book of Hoffstein, Pipher and Silverman.

For random $a$ the probability for it not being a witness is 25%, therefore the probability of not finding 10 witnesses at random if $a$ is composite is $0.25^{10}$, i.e. If no witnesses are found in 10 rounds, then the probability for $a$ being prime is $1-0.25^{10}$= 0.999999

In [11]:
for n in range(1, 2000):
    if isPrime(2**n-1, 100):
        print(n)

2
3
5
7
13
17
19
31
61
89
107
127
521
607
1279


In [12]:
2**607-1

531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728127

In [13]:
len(str(2**607-1))

183

In [14]:
len(bin(2**607-1))

609